# Topology metrics vs threshold and environmental drivers

Metrics: Connectivity, Flow_Efficiency, Degree_Centrality, Stream_Order

Outputs (PDF):
- Metric vs threshold (th=0..34)
- Metric vs env vars at th=0 and th=15


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip -q install rasterio whitebox

In [ ]:
import os
import numpy as np
import pandas as pd
import rasterio
import networkx as nx
import matplotlib.pyplot as plt
import whitebox
from tqdm import tqdm

## Config

In [ ]:
DRIVE_ROOT = '/content/drive/My Drive'
PROJECT_DIR = os.path.join(DRIVE_ROOT, 'Oyster_Flow')
RESULT_DIR = os.path.join(PROJECT_DIR, 'OFG_result')
os.makedirs(RESULT_DIR, exist_ok=True)

# Hydrology rasters
flow_accum = os.path.join(PROJECT_DIR, 'FlowAcc_Flow51.tif')
flow_dir = os.path.join(PROJECT_DIR, 'FlowDir_Fill31.tif')

streams_rast = os.path.join(RESULT_DIR, 'HS_slope_streams.tif')
stream_link_r = os.path.join(RESULT_DIR, 'HS_slope_link.tif')
stream_order_r = os.path.join(RESULT_DIR, 'HS_slope_order.tif')

# Env layers
base_dir = os.path.join(PROJECT_DIR, 'OFG_New')
env_vars = ['w1', 'v1', 'temp1', 'NO32', 'chlorophyll1', 'salt1']
display_names = {'NO32': 'NO$_3$2'}

# Thresholds
THRESHOLDS = list(range(35))

wbt = whitebox.WhiteboxTools()
wbt.set_working_dir(RESULT_DIR)

## Helpers

In [ ]:
def load_env_arrays(var_list, base_directory):
    arrays = {}
    for v in var_list:
        path = os.path.join(base_directory, f'{v}.tif')
        with rasterio.open(path) as src:
            arr = src.read(1).astype(np.float32)
            if src.nodata is not None:
                arr = np.where(arr == src.nodata, np.nan, arr)
        arrays[v] = arr
    return arrays

def extract_network(threshold):
    wbt.extract_streams(flow_accum, streams_rast, threshold)
    wbt.stream_link_class(d8_pntr=flow_dir, streams=streams_rast, output=stream_link_r)
    wbt.strahler_stream_order(d8_pntr=flow_dir, streams=streams_rast, output=stream_order_r)

    with rasterio.open(streams_rast) as src:
        streams_data = src.read(1).astype(np.float32)
    with rasterio.open(flow_accum) as src:
        flow_acc_data = src.read(1).astype(np.float32)
    with rasterio.open(stream_order_r) as src:
        stream_order_data = src.read(1).astype(np.float32)

    flow_acc_data[flow_acc_data < 0] = 0

    graph = nx.Graph()
    rows, cols = np.where(streams_data > 0)
    for r, c in zip(rows, cols):
        graph.add_node((r, c), flow_acc=flow_acc_data[r, c], stream_order=stream_order_data[r, c])
    for r, c in zip(rows, cols):
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < streams_data.shape[0] and 0 <= nc < streams_data.shape[1]:
                if streams_data[nr, nc] > 0:
                    graph.add_edge((r, c), (nr, nc))
    return graph

def binned_curve(df, x_col, y_col, n_bins=20):
    x = df[x_col].values
    y = df[y_col].values
    mask = np.isfinite(x) & np.isfinite(y)
    x = x[mask]
    y = y[mask]
    if x.size == 0:
        return np.array([]), np.array([])
    bins = np.linspace(np.min(x), np.max(x), n_bins + 1)
    inds = np.digitize(x, bins)
    centers = []
    means = []
    for i in range(1, len(bins)):
        sel = y[inds == i]
        if sel.size > 0:
            centers.append((bins[i-1] + bins[i]) / 2)
            means.append(np.mean(sel))
    return np.array(centers), np.array(means)

def metric_values_for_graph(graph):
    if graph.number_of_nodes() == 0:
        return {
            'Flow_Efficiency': 0,
            'Degree_Centrality': 0,
            'Stream_Order': 0,
        }, {}, {}
    connectivity = nx.number_connected_components(graph)
    flow_eff = float(np.mean([d['flow_acc'] for _, d in graph.nodes(data=True)]))
    deg_cent = nx.degree_centrality(graph)
    avg_deg = float(np.mean(list(deg_cent.values())))
    avg_order = float(np.mean([d['stream_order'] for _, d in graph.nodes(data=True)]))
    return {
        'Flow_Efficiency': flow_eff,
        'Degree_Centrality': avg_deg,
        'Stream_Order': avg_order,
    }, deg_cent, None

def node_dataframe_for_metric(graph, env_arrays, metric, deg_cent=None):
    rows = []
    if graph.number_of_nodes() == 0:
        return pd.DataFrame()
    if metric == 'Degree_Centrality' and deg_cent is None:
        deg_cent = nx.degree_centrality(graph)
    for (r, c), attrs in graph.nodes(data=True):
        row = {}
        if metric == 'Connectivity':
            row[metric] = nx.number_connected_components(graph)
        elif metric == 'Flow_Efficiency':
            row[metric] = float(np.mean([d['flow_acc'] for _, d in graph.nodes(data=True)]))
        elif metric == 'Degree_Centrality':
            row[metric] = float(deg_cent.get((r, c), 0))
        elif metric == 'Stream_Order':
            row[metric] = float(attrs.get('stream_order', 0))
        for v, arr in env_arrays.items():
            try:
                row[v] = float(arr[r, c])
            except Exception:
                row[v] = np.nan
        rows.append(row)
    df = pd.DataFrame(rows).dropna()
    return df


## 1) Metric vs threshold (th=0..34)

In [ ]:
metrics = ['Flow_Efficiency', 'Degree_Centrality', 'Stream_Order']
rows = []
for th in tqdm(THRESHOLDS, desc='thresholds'):
    g = extract_network(th)
    metric_vals, deg_cent, _ = metric_values_for_graph(g)
    for m in metrics:
        rows.append({'threshold': th, 'metric': m, 'value': metric_vals[m]})

df = pd.DataFrame(rows)
for m in metrics:
    sub = df[df['metric'] == m]
    plt.figure(figsize=(8, 5))
    plt.plot(sub['threshold'], sub['value'], marker='o')
    plt.xlabel('threshold')
    plt.ylabel(m)
    plt.title(f'{m} vs threshold')
    plt.grid(True)
    out_pdf = os.path.join(RESULT_DIR, f'{m}_vs_threshold.pdf')
    plt.savefig(out_pdf, bbox_inches='tight')
    plt.show()


In [ ]:
import os
import numpy as np
import pandas as pd
import rasterio
import networkx as nx
import matplotlib.pyplot as plt
import whitebox
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

In [ ]:
DRIVE_ROOT = '/content/drive/My Drive'
PROJECT_DIR = os.path.join(DRIVE_ROOT, 'Oyster_Flow')
RESULT_DIR = os.path.join(PROJECT_DIR, 'OFG_result')
os.makedirs(RESULT_DIR, exist_ok=True)

flow_accum = os.path.join(PROJECT_DIR, 'FlowAcc_Flow51.tif')
flow_dir = os.path.join(PROJECT_DIR, 'FlowDir_Fill31.tif')

streams_rast = os.path.join(RESULT_DIR, 'HS_slope_streams.tif')
stream_order_r = os.path.join(RESULT_DIR, 'HS_slope_order.tif')

base_dir = os.path.join(PROJECT_DIR, 'OFG_New')
env_vars = ['w1', 'v1', 'temp1', 'NO32', 'chlorophyll1', 'salt1']
display_names = {'NO32': 'NO$_3$2'}

basin_shp = os.path.join(PROJECT_DIR, 'OFG_New', 'oysterbasin0325.shp')

THRESHOLDS = list(range(35))

wbt = whitebox.WhiteboxTools()
wbt.set_working_dir(RESULT_DIR)

In [ ]:
def load_env_arrays(var_list, base_directory):
    arrays = {}
    for v in var_list:
        path = os.path.join(base_directory, f'{v}.tif')
        with rasterio.open(path) as src:
            arr = src.read(1).astype(np.float32)
            if src.nodata is not None:
                arr = np.where(arr == src.nodata, np.nan, arr)
        arrays[v] = arr
    return arrays
def extract_graph(threshold):
    wbt.extract_streams(flow_accum, streams_rast, threshold)
    wbt.strahler_stream_order(d8_pntr=flow_dir, streams=streams_rast, output=stream_order_r)
    with rasterio.open(streams_rast) as src:
        streams_data = src.read(1).astype(np.float32)
        transform = src.transform
        streams_crs = src.crs
    with rasterio.open(flow_accum) as src:
        flow_acc_data = src.read(1).astype(np.float32)
    with rasterio.open(stream_order_r) as src:
        stream_order_data = src.read(1).astype(np.float32)
    flow_acc_data[flow_acc_data < 0] = 0
    graph = nx.Graph()
    rows, cols = np.where(streams_data > 0)
    for r, c in zip(rows, cols):
        graph.add_node((r, c), flow_acc=flow_acc_data[r, c], stream_order=stream_order_data[r, c])
    for r, c in zip(rows, cols):
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < streams_data.shape[0] and 0 <= nc < streams_data.shape[1]:
                if streams_data[nr, nc] > 0:
                    graph.add_edge((r, c), (nr, nc))
    return graph, transform, streams_crs
def assign_basins(graph, transform, streams_crs, basins):
    node_list = []
    for node in graph.nodes():
        r, c = node
        lon, lat = rasterio.transform.xy(transform, r, c)
        node_list.append({'node': node, 'lon': lon, 'lat': lat})
    node_df = pd.DataFrame(node_list)
    node_gdf = gpd.GeoDataFrame(
        node_df,
        geometry=[Point(xy) for xy in zip(node_df['lon'], node_df['lat'])],
        crs=streams_crs
    )
    node_basins = gpd.sjoin(node_gdf, basins, how='left', predicate='within')
    if 'basin_id' not in node_basins.columns:
        node_basins['basin_id'] = node_basins.index_right
    node_basins = node_basins.dropna(subset=['basin_id'])
    node_basins['basin_id'] = node_basins['basin_id'].astype(int)
    return node_basins
def basin_metrics_df(graph, node_basins):
    rows = []
    for bid in node_basins['basin_id'].unique():
        subset = node_basins[node_basins['basin_id'] == bid]
        nodes_in_basin = subset['node'].tolist()
        subgraph = graph.subgraph(nodes_in_basin)
        if len(subgraph) < 2:
            continue
        connectivity = nx.number_connected_components(subgraph)
        max_order = int(np.max([d['stream_order'] for _, d in subgraph.nodes(data=True)]))
        rows.append({
            'basin_id': bid,
            'Connectivity': connectivity,
            'centroid_lon': subset['lon'].mean(),
            'centroid_lat': subset['lat'].mean(),
        })
    return pd.DataFrame(rows)
def attach_basin_env_means(basin_df, node_basins, env_arrays):
    # compute basin-level mean env values using node locations
    env_rows = []
    for bid in basin_df['basin_id'].unique():
        subset = node_basins[node_basins['basin_id'] == bid]
        if subset.empty:
            continue
        vals = {'basin_id': bid}
        for v, arr in env_arrays.items():
            vs = []
            for node in subset['node']:
                r, c = node
                try:
                    vs.append(float(arr[r, c]))
                except Exception:
                    pass
            vals[v] = float(np.nanmean(vs)) if vs else np.nan
        env_rows.append(vals)
    env_df = pd.DataFrame(env_rows)
    return basin_df.merge(env_df, on='basin_id', how='left')
def binned_curve(df, x_col, y_col, n_bins=20):
    x = df[x_col].values
    y = df[y_col].values
    mask = np.isfinite(x) & np.isfinite(y)
    x = x[mask]
    y = y[mask]
    if x.size == 0:
        return np.array([]), np.array([])
    bins = np.linspace(np.min(x), np.max(x), n_bins + 1)
    inds = np.digitize(x, bins)
    centers = []
    means = []
    for i in range(1, len(bins)):
        sel = y[inds == i]
        if sel.size > 0:
            centers.append((bins[i-1] + bins[i]) / 2)
            means.append(np.mean(sel))
    return np.array(centers), np.array(means)


In [ ]:
def basin_metrics_df(graph, node_basins):
    rows = []
    for bid in node_basins['basin_id'].unique():
        subset = node_basins[node_basins['basin_id'] == bid]
        nodes_in_basin = subset['node'].tolist()
        subgraph = graph.subgraph(nodes_in_basin)
        if len(subgraph) < 2:
            continue
        connectivity = nx.number_connected_components(subgraph)
        stream_orders = [d['stream_order'] for _, d in subgraph.nodes(data=True) if 'stream_order' in d]
        max_order = int(np.max(stream_orders)) if stream_orders else 0
        rows.append({
            'basin_id': bid,
            'Connectivity': connectivity,
            'centroid_lon': subset['lon'].mean(),
            'centroid_lat': subset['lat'].mean(),
        })
    return pd.DataFrame(rows)

basins = gpd.read_file(basin_shp)
rows = []
for th in tqdm(THRESHOLDS, desc='thresholds'):
    graph, transform, streams_crs = extract_graph(th)
    node_basins = assign_basins(graph, transform, streams_crs, basins)
    bdf = basin_metrics_df(graph, node_basins)
    if bdf.empty:
        rows.append({'threshold': th, 'metric': 'Connectivity', 'value': np.nan})
        continue
    rows.append({'threshold': th, 'metric': 'Connectivity', 'value': bdf['Connectivity'].mean()})

df = pd.DataFrame(rows)
for m in ['Connectivity']:
    sub = df[df['metric'] == m]
    plt.figure(figsize=(8, 5))
    plt.plot(sub['threshold'], sub['value'], marker='o')
    plt.xlabel('threshold')
    plt.ylabel(m if m != 'Connectivity' else 'Connectivity (components)')
    plt.title(f'Basin-mean {m} vs threshold' if m != 'Connectivity' else 'Basin-mean Connectivity (components) vs threshold')
    plt.grid(True)
    out_pdf = os.path.join(RESULT_DIR, f'basin_{m}_vs_threshold.pdf')
    plt.savefig(out_pdf, bbox_inches='tight')
    plt.show()
